In [1]:
import numpy as np
import pandas as pd
import re
import os
import random
import pprint
from collections import defaultdict

In [4]:
def remove_nan(df:pd.DataFrame) -> dict:
    
    """
    Rimuove i valori nulli da una lista
    """

    lookup_dict = df.to_dict('list')

    for k, v in lookup_dict.items():

        while np.nan in lookup_dict[k]:
            lookup_dict[k].remove(np.nan)

    return lookup_dict

In [2]:
df_entities = pd.read_excel("dataset.xlsx", sheet_name='entities_slots')

In [5]:
lookups =  remove_nan(df_entities)
lookups

{'SEDE': ['EBOLI',
  'SOMMA VESUVIANA',
  'NAPOLI',
  'CASERTA',
  'SALERNO',
  'BENEVENTO',
  'AVELLINO',
  'VICO EQUENSE',
  'ASCEA',
  'ACERNO',
  'ALVIGNANO',
  'PASTORANO',
  'ROCCABASCERANA',
  'TRAMONTI',
  'MONTECORVINO ROVELLA',
  'CAPACCIO',
  'NOCERA INFERIORE',
  'MONTECALVO IRPINO',
  'CIORLANO',
  'MONTELLA',
  'MONTECORVINO PUGLIANO'],
 'PROVINCIA': ['NAPOLI', 'CASERTA', 'SALERNO', 'BENEVENTO', 'AVELLINO'],
 'CATEGORIA': ['ALLEVAMENTO', 'FRUTTA', 'FORMAGGI']}

In [9]:
df_user = pd.read_excel('dataset.xlsx',sheet_name='user', header=None)
df_user.columns = ['user','sentences']
df_user['user'] = df_user['user'].fillna(method='ffill', axis=0)

sentences = defaultdict(list)

df_grouped = df_user.groupby('user')

for group in df_grouped.groups:
    
    sentences[group] = df_grouped.get_group(group)['sentences'].tolist()

In [10]:
sentences

defaultdict(list,
            {'user_AskCity': ['Ci sono fattorie didattiche ad [SEDE](Eboli)?',
              'Quante fattorie didattiche ci sono a [SEDE](Salerno)?'],
             'user_AskProv': ['Quante fattorie didattiche ci sono in provincia di [PROVINCIA](Salerno)?',
              'Ci sono fattorie didattiche in provincia di [PROVINCIA](Salerno)?',
              'Ci sono fattorie didattiche nella provincia di [PROVINCIA](Napoli)?'],
             'user_AskProvWithParams': ['Quante fattorie didattiche ci sono in provincia di [PROVINCIA](Salerno) con [categoria](allevamento)?',
              'Ci sono fattorie didattiche in provincia di [PROVINCIA](Salerno) con produzione di [CATEGORIA](formaggi)?',
              'Ci sono fattorie didattiche nella provincia di [PROVINCIA](Napoli) che producono [CATEGORIA](formaggi)?',
              'Ci sono fattorie didattiche nella provincia di [PROVINCIA](Napoli) con coltivazione di [CATEGORIA](frutta)?']})

In [11]:
df_bot = pd.read_excel('dataset.xlsx',sheet_name='bot', header=None)
df_bot.columns = ['bot','sentences']
df_bot['bot'] = df_bot['bot'].fillna(method='ffill', axis=0)

bot_sentences = defaultdict(list)

df_grouped = df_bot.groupby('bot')

for group in df_grouped.groups:
    
    bot_sentences[group] = df_grouped.get_group(group)['sentences'].tolist()
    
bot_sentences

defaultdict(list,
            {'bot_ReplyCity ': ['Nella città di [SEDE] ci sono %d fattorie didattiche'],
             'bot_ReplyProv': ['Nella provincia di [PROVINCIA] ci sono %d fattorie didattiche'],
             'bot_ReplyProvWithParams': ['Nella provincia di [PROVINCIA] ci sono %d fattorie didattiche con le caratteristiche richieste']})

In [12]:
df_dialogs = pd.read_excel('dataset.xlsx',sheet_name='dialogs')
dialogs = remove_nan(df_dialogs) 
dialogs

{'Dialog_1': ['user_AskProv', 'bot_ReplyProv'],
 'Dialog_2': ['user_AskCity', 'bot_ReplyCity '],
 'Dialog_3': ['user_AskProvWithParams', 'bot_ReplyProvWithParams']}

In [13]:
sentences_file = "sentences_origin.txt"

i = True

for k, v in sentences.items():
    
    for values in v:
        
        if i:
            with open(sentences_file, 'w+') as f:
                f.writelines(f'{values}|{k}\n')
                i = False
        else:
            with open(sentences_file, 'a+') as f:
                f.writelines(f'{values}|{k}\n')

In [14]:
sentences, categories = [], []

with open(sentences_file, encoding='utf-8') as f:
    dataset = f.read()
    dataset = dataset.split("\n")

for data in dataset:
    sentence = data.split("|")
    
    if len(sentence) > 1:
        
        sentences.append(sentence[0].upper())
        categories.append(sentence[1])
        
assert len(sentences) == len(categories)

In [15]:
sentences[0]

'CI SONO FATTORIE DIDATTICHE AD [SEDE](EBOLI)?'

In [16]:
categories[0]

'user_AskCity'

In [18]:
sentences_file_generated = "sentences_generated.txt"

n_sentences = 1000

In [21]:
slots = list(lookups.keys())

slots

['SEDE', 'PROVINCIA', 'CATEGORIA']

In [22]:
for i in range(n_sentences):
        
    index = random.randint(0, len(sentences)-1)

    sentence = sentences[index]
    category = categories[index]
    
    for key in slots:
        
        """
        Ogni volta che regex individua lo slot nella frase
            sostituisce il valore con uno estratto in modo casuale
        """
        
        regex_str = fr'\[{key}]\((?P<value>[a-z ]+)\)+'
        
        slot_match = re.compile(regex_str)

        repl = f"[{key}]({random.choice(lookups[key])})"

        sentence = slot_match.sub(repl, sentence)
        
        
        """
        Poi riassocia la categoria di partenza
        """
        
    if i == 0:
        
        with open(sentences_file_generated, 'w+') as f:
            f.writelines(f'{sentence}|{category}\n')
        
    else:
    
        with open(sentences_file_generated, 'a+') as f:
            f.writelines(f'{sentence}|{category}\n')

In [23]:
def rimuovi_punteggiatura(text):
    
    """
    I dati di training contengono parentesi quadre e tonde, 
        quindi non vanno eliminate in questa fase
    """
    
    text = re.sub(r'[\.,;:!?]' , " ", text)
    text = re.sub(r'\s+' , " ", text)
    
    return text

In [24]:
with open(sentences_file_generated, encoding='utf-8') as f:
    sentences = f.read()
    sentences = sentences.split("\n")

In [25]:
example = random.choice(sentences)
example, _ = example.split("|")
print("Before:",example)

example = re.sub(r'\[(?P<name>[a-zA-Z_]+)\]|\(|\)+', "", example)
print("After:",example)

Before: CI SONO FATTORIE DIDATTICHE IN PROVINCIA DI [PROVINCIA](SALERNO) CON PRODUZIONE DI [CATEGORIA](FORMAGGI)?
After: CI SONO FATTORIE DIDATTICHE IN PROVINCIA DI SALERNO CON PRODUZIONE DI FORMAGGI?


In [26]:
sentence = sentences[10]
print(f"Fase iniziale: {sentence} \n")

sentence, categ = sentence.split("|")
sentence = rimuovi_punteggiatura(sentence)

print(f"Fase finale: {sentence}")

Fase iniziale: CI SONO FATTORIE DIDATTICHE IN PROVINCIA DI [PROVINCIA](SALERNO)?|user_AskProv 

Fase finale: CI SONO FATTORIE DIDATTICHE IN PROVINCIA DI [PROVINCIA](SALERNO) 


In [27]:
regex_str = r'\[(?P<name>[a-zA-Z_]+)\]\((?P<value>[a-zA-Z\' ]+)\)+'
slot_match = re.compile(regex_str)

In [28]:
splits = slot_match.split(sentence)

splits

['CI SONO FATTORIE DIDATTICHE IN PROVINCIA DI ', 'PROVINCIA', 'SALERNO', ' ']

In [29]:
matches = slot_match.findall(sentence)

dct = {k:v for k,v in matches}
dct

{'PROVINCIA': 'SALERNO'}

In [31]:
for split in splits:
   
    if split in list(dct.values()):
        for value in split.split():
            
            index = list(dct.values()).index(split)
            key = list(dct.keys())[index]
            
            print(value, "->", key)
    elif split in list(dct.keys()):
        pass
    else:
        for splt in split.split():
            print(splt, "->", '0')

CI -> 0
SONO -> 0
FATTORIE -> 0
DIDATTICHE -> 0
IN -> 0
PROVINCIA -> 0
DI -> 0
SALERNO -> PROVINCIA


In [32]:
arr_sentences = list()
arr_categories = list()

for n, sentence in enumerate(sentences):

    try:
    
        sentence, categ = sentence.split("|")
        
        sentence = rimuovi_punteggiatura(sentence)

        arr_categories.append(categ)

        splits = slot_match.split(sentence)

#         match = slot_match.search(frase)
        matches = slot_match.findall(sentence)
    
        dct = {k:v for k,v in matches}        

        if matches is not None:
            
            for split in splits:
                if split in list(dct.values()):
                    for value in split.split():
                        
                        index = list(dct.values()).index(split)
                        key = list(dct.keys())[index]
                        
                        arr_sentences.append([n, value, key])
                elif split in list(dct.keys()):
                    pass
                else:
                    for value in split.split():
                        arr_sentences.append([n, value, 'O'])
                        
        else:
            """
            Serve per verificare se in qualche frase non avviene il match
            """
            print(n, frase)
            
    except Exception as err:
        pass


arr_sentences[:10]

[[0, 'QUANTE', 'O'],
 [0, 'FATTORIE', 'O'],
 [0, 'DIDATTICHE', 'O'],
 [0, 'CI', 'O'],
 [0, 'SONO', 'O'],
 [0, 'IN', 'O'],
 [0, 'PROVINCIA', 'O'],
 [0, 'DI', 'O'],
 [0, 'SALERNO', 'PROVINCIA'],
 [0, 'CON', 'O']]

In [34]:
df = pd.DataFrame(arr_sentences, columns=['n_frase','word','tag'])
df.head(15)

,n_frase,word,tag
0,0,QUANTE,O
1,0,FATTORIE,O
2,0,DIDATTICHE,O
3,0,CI,O
4,0,SONO,O
5,0,IN,O
6,0,PROVINCIA,O
7,0,DI,O
8,0,SALERNO,PROVINCIA
9,0,CON,O


In [35]:
df_target = df[['n_frase','tag']]
df_target.head(10)

,n_frase,tag
0,0,O
1,0,O
2,0,O
3,0,O
4,0,O
5,0,O
6,0,O
7,0,O
8,0,PROVINCIA
9,0,O


In [37]:
def prepare_target_crf(df:pd.DataFrame) -> list:
    
    """
    Ultimo step per preparare i target per addestrare l'algoritmo
    
    Parameters:
    -----------
    
    df : pd.DataFrame
    
        il DataFrame deve contenere due colonne, una che indicizza la frase
        e l'altra che indica se il valore è uno slot o meno
    
    
    Returns:
    -----------
    y : list
    
        una lista annidata
    
    """
    
    y = list()

    for k, v in df.groupby('n_frase'):

        y.append(v['tag'].tolist())
    
    return y

In [38]:
y = prepare_target_crf(df_target)
y[:5]

[['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'PROVINCIA', 'O', 'CATEGORIA'],
 ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'PROVINCIA'],
 ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'PROVINCIA', 'O', 'O', 'CATEGORIA'],
 ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'PROVINCIA', 'O', 'O', 'O', 'CATEGORIA'],
 ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'PROVINCIA', 'O', 'O', 'O', 'CATEGORIA']]

In [39]:
df_sample = df.head(100).copy()

In [45]:
import spacy

nlp = spacy.load("it_core_news_lg")

In [46]:
def spacy_entities_extractor(txt):
    
    doc = nlp(txt.capitalize())
    token = doc[0]
    
    if token.ent_type_ != '':
        return token.ent_type_ # pos_
    else:
        return 'O'

In [47]:
df_sample['shift-3'] = df_sample.groupby('n_frase')['word'].shift(1).str.slice(-3)
df_sample['shift+3'] = df_sample.groupby('n_frase')['word'].shift(-1).str.slice(0,3)

df_sample['shift-3'].fillna('BOF', inplace=True)
df_sample['shift+3'].fillna('EOF', inplace=True)

df_sample['spacy'] = df_sample['word'].apply(spacy_entities_extractor)

df_sample['bias'] = 1

df_sample

,n_frase,word,tag,shift-3,shift+3,bias,spacy
0,0,QUANTE,O,BOF,FAT,1,O
1,0,FATTORIE,O,NTE,DID,1,O
2,0,DIDATTICHE,O,RIE,CI,1,O
3,0,CI,O,CHE,SON,1,O
4,0,SONO,O,CI,IN,1,O
...,...,...,...,...,...,...,...
95,9,IN,O,ONO,PRO,1,O
96,9,PROVINCIA,O,IN,DI,1,O
97,9,DI,O,CIA,SAL,1,O
98,9,SALERNO,PROVINCIA,DI,EOF,1,LOC


In [48]:
def extend_data(df:pd.DataFrame, spacy:bool=False) -> pd.DataFrame:
    
    """
    Estende i dati attraverso un algortimo personalizzato
    """
    
    df['shift-3'] = df.groupby('n_frase')['word'].shift(1).str.slice(-3)
    df['shift+3'] = df.groupby('n_frase')['word'].shift(-1).str.slice(0,3)
    df['shift-10'] = df.groupby('n_frase')['word'].shift(1).str.slice(-10)
    df['shift+10'] = df.groupby('n_frase')['word'].shift(-1).str.slice(0,10)

    df['shift-3'].fillna('BOF', inplace=True)
    df['shift+3'].fillna('EOF', inplace=True)
    df['shift-10'].fillna('BOF', inplace=True)
    df['shift+10'].fillna('EOF', inplace=True)
    
    if spacy:
        df['spacy'] = df['word'].apply(spacy_entities_extractor)

    df['bias'] = 1
    
    try:
        df.drop(columns=['tag'], inplace=True)
    except:
        pass

    
    return df

In [49]:
df = extend_data(df, spacy=False)
df.head()

,n_frase,word,shift-3,shift+3,shift-10,shift+10,bias
0,0,QUANTE,BOF,FAT,BOF,FATTORIE,1
1,0,FATTORIE,NTE,DID,QUANTE,DIDATTICHE,1
2,0,DIDATTICHE,RIE,CI,FATTORIE,CI,1
3,0,CI,CHE,SON,DIDATTICHE,SONO,1
4,0,SONO,CI,IN,CI,IN,1


In [50]:
X = list()

for k, v in df.groupby('n_frase'):
    
    v.drop(columns='n_frase', inplace=True)

    X.append(v.to_dict('records'))

In [51]:
X[0]

[{'word': 'QUANTE',
  'shift-3': 'BOF',
  'shift+3': 'FAT',
  'shift-10': 'BOF',
  'shift+10': 'FATTORIE',
  'bias': 1},
 {'word': 'FATTORIE',
  'shift-3': 'NTE',
  'shift+3': 'DID',
  'shift-10': 'QUANTE',
  'shift+10': 'DIDATTICHE',
  'bias': 1},
 {'word': 'DIDATTICHE',
  'shift-3': 'RIE',
  'shift+3': 'CI',
  'shift-10': 'FATTORIE',
  'shift+10': 'CI',
  'bias': 1},
 {'word': 'CI',
  'shift-3': 'CHE',
  'shift+3': 'SON',
  'shift-10': 'DIDATTICHE',
  'shift+10': 'SONO',
  'bias': 1},
 {'word': 'SONO',
  'shift-3': 'CI',
  'shift+3': 'IN',
  'shift-10': 'CI',
  'shift+10': 'IN',
  'bias': 1},
 {'word': 'IN',
  'shift-3': 'ONO',
  'shift+3': 'PRO',
  'shift-10': 'SONO',
  'shift+10': 'PROVINCIA',
  'bias': 1},
 {'word': 'PROVINCIA',
  'shift-3': 'IN',
  'shift+3': 'DI',
  'shift-10': 'IN',
  'shift+10': 'DI',
  'bias': 1},
 {'word': 'DI',
  'shift-3': 'CIA',
  'shift+3': 'SAL',
  'shift-10': 'PROVINCIA',
  'shift+10': 'SALERNO',
  'bias': 1},
 {'word': 'SALERNO',
  'shift-3': 'DI',
  

In [52]:
y[0]

['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'PROVINCIA', 'O', 'CATEGORIA']

In [53]:
def prepare_data_crf(df:pd.DataFrame) -> list:
    
    """
    Ultimo step per preparare i dati per addestrare l'algoritmo
    
    Parameters:
    -----------
    
    df : pd.DataFrame
    
        il numero di colonne del DataFrame dipende da come è stata impostata
        la Data Augmentation... l'importante è che il df contenga
        la colonna 'n_frase' usata come indice
    
    
    Returns:
    -----------
    y : list
    
        una lista annidata di dictionary
    
    """

    
    X = list()

    for k, v in df.groupby('n_frase'):

        v.drop(columns='n_frase', inplace=True)

        X.append(v.to_dict('records'))
    
    return X

In [54]:
X = prepare_data_crf(df)

In [55]:
import sklearn_crfsuite

In [56]:
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=100,
    all_possible_states=False,     # Default
    all_possible_transitions=False # Default
)

In [57]:
crf.fit(X, y)

/home/ubuntupc/.local/lib/python3.8/site-packages/sklearn/base.py:193: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  warnings.warn('From version 0.24, get_params will raise an '


CRF(algorithm='lbfgs', all_possible_states=False,
    all_possible_transitions=False, averaging=None, c=None, c1=0.1, c2=0.1,
    calibration_candidates=None, calibration_eta=None,
    calibration_max_trials=None, calibration_rate=None,
    calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
    gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=100,
    max_linesearch=None, min_freq=None, model_filename=None, num_memories=None,
    pa_type=None, period=None, trainer_cls=None, variance=None, verbose=False)

In [58]:
sentences[5].split("|")[0]

'CI SONO FATTORIE DIDATTICHE IN PROVINCIA DI [PROVINCIA](SALERNO) CON PRODUZIONE DI [CATEGORIA](FORMAGGI)?'

In [59]:
for n, word in enumerate(X[5]):
    
    print(f"{n} --> {word['word']}")

0 --> CI
1 --> SONO
2 --> FATTORIE
3 --> DIDATTICHE
4 --> IN
5 --> PROVINCIA
6 --> DI
7 --> SALERNO
8 --> CON
9 --> PRODUZIONE
10 --> DI
11 --> FORMAGGI


In [60]:
y_pred = crf.predict_single(X[5])

for n, pred in enumerate(y_pred):
    
    print(f"{n} --> {pred}")

0 --> O
1 --> O
2 --> O
3 --> O
4 --> O
5 --> O
6 --> O
7 --> PROVINCIA
8 --> O
9 --> O
10 --> O
11 --> CATEGORIA


In [61]:
def prepare_sentence(sentence:str) -> [pd.DataFrame, list]:
    
    """
    Prepare la frase per il predict
    
    Parameters:
    -----------
    
    sentence : str
    
        è la frase che sarà elaborata
    
    
    Returns:
    -----------
    df : DataFrame
    
    X_arr[0] : array
        
        dati in formato utile a CRF per il predict
    
    """

    sentence = rimuovi_punteggiatura(sentence)
    
    X_arr = list()
    
    df = pd.DataFrame(data = [i for i in sentence.split()], columns=['word'])
    df['n_frase'] = 1
    
    df = extend_data(df)
    
    
    for k, v in df.groupby('n_frase'):

        v.drop(columns='n_frase', inplace=True)

        X_arr.append(v.to_dict('records'))
    
    
    return df, X_arr[0]

In [62]:
def extend_sentence(sentence:str, model:sklearn_crfsuite.estimator.CRF) -> pd.DataFrame:
    
   
    """
    Estrae slots e lo aggiunge al DataFrame come colonna
    
    Parameters:
    -----------
    
    sentence : str
    
        è la stringa che contiene la frase
        
    model : sklearn_crfsuite.estimator.CRF
        
        è il modello addestrato di ConditionalRandomField
    
    
    Returns:
    -----------
    df : DataFrame
    
        al DataFrame di partenza viene aggiunta una colonna 
        con l'indicazione del tipo di slot individuato
       
    """
    
    
    df, X_arr = prepare_sentence(sentence)
    
    df['slots'] = model.predict_single(X_arr)
    
    return df

In [64]:
nuova_frase = 'CI SONO FATTORIE DIDATTICHE AD AFRAGOLA'

In [65]:
extend_sentence(nuova_frase, crf)

,word,n_frase,shift-3,shift+3,shift-10,shift+10,bias,slots
0,CI,1,BOF,SON,BOF,SONO,1,O
1,SONO,1,CI,FAT,CI,FATTORIE,1,O
2,FATTORIE,1,ONO,DID,SONO,DIDATTICHE,1,O
3,DIDATTICHE,1,RIE,AD,FATTORIE,AD,1,O
4,AD,1,CHE,AFR,DIDATTICHE,AFRAGOLA,1,O
5,AFRAGOLA,1,AD,EOF,AD,EOF,1,SEDE


## Duckling

[datetime.datetime(2021, 11, 21, 8, 0, tzinfo=<StaticTzInfo 'UTC\-08:00'>)]}

In [66]:
def slots_extractor(sentence:str, model:sklearn_crfsuite.estimator.CRF) -> defaultdict:
    
    
    """
    Restituisce un dictionary degli slots individuati
    
    Parameters:
    -----------
    
    sentence : str
    
        è la stringa che contiene la frase
        
    model : sklearn_crfsuite.estimator.CRF
        
        è il modello addestrato di ConditionalRandomField
    
    
    Returns:
    -----------
    dd : dictionary
           
    """
    
    df = extend_sentence(sentence, model)
    
    dd = defaultdict(list)

    for k, v in df.query("slots != 'O'").groupby('slots'):
        dd[k] = " ".join(v['word'])
        
    
    
    """
    Estrazione date ed orari tramite duckling    
    """
    
    date_time = extract_datetime(sentence)

    if date_time is not None and len(date_time) > 0:
        dd['DATETIMES'] = date_time['datetime']
#     else:
#         sentence_dict['datetimes'] = False


    return dd

In [67]:
df = pd.read_csv(sentences_file_generated, sep="|", header=None)

df.columns = ["sentences", "intents"]
df.head()

,sentences,intents
0,QUANTE FATTORIE DIDATTICHE CI SONO IN PROVINCI...,user_AskProvWithParams
1,QUANTE FATTORIE DIDATTICHE CI SONO IN PROVINCI...,user_AskProv
2,CI SONO FATTORIE DIDATTICHE NELLA PROVINCIA DI...,user_AskProvWithParams
3,CI SONO FATTORIE DIDATTICHE IN PROVINCIA DI [P...,user_AskProvWithParams
4,CI SONO FATTORIE DIDATTICHE IN PROVINCIA DI [P...,user_AskProvWithParams


In [69]:
def conserva_solo_slot_name(text: str) -> str:

    """
    Per facilitare la riduzione dello spazio dimensionale
        vengono eliminati i valori degli slots
        mentre vengono conservati i loro nomi


    Parameters:
    -----------

    text : str

        è la stringa che contiene la frase


    Returns:
    -----------
    text : str

    """

    text = rimuovi_punteggiatura(text)

    pattern = r"(\([A-Za-z0-9 ]+\)|\[|\])"

    text = re.sub(pattern, "", text)

    return text

In [70]:
df["sentences"][5]

'CI SONO FATTORIE DIDATTICHE IN PROVINCIA DI [PROVINCIA](SALERNO) CON PRODUZIONE DI [CATEGORIA](FORMAGGI)?'

In [71]:
conserva_solob_slot_name(df["sentences"][5])

'CI SONO FATTORIE DIDATTICHE IN PROVINCIA DI PROVINCIA CON PRODUZIONE DI CATEGORIA '

In [74]:
df["sentences"] = df["sentences"].apply(conserva_solo_slot_name)

df.sentences[0]

'QUANTE FATTORIE DIDATTICHE CI SONO IN PROVINCIA DI PROVINCIA CON CATEGORIA '

In [75]:
def replace_slot_values(sentence_dict: dict) -> dict:

    """
    Integrazione dictionary - parte 1 di 2


    Per operare una riduzione delle variabili
    sostituisce il valore con il relativo slot
    """

    sentence_dict["replaced_sentence"] = sentence_dict["sentence"]

    for k, v in sentence_dict["slots"].items():

        if k != "DATETIMES":

            sentence_dict["replaced_sentence"] = re.sub(
                v, k, sentence_dict["replaced_sentence"]
            )

    return sentence_dict

In [79]:
import requests
import json
import datetime
import dateparser

def extract_datetime(text:str, url = "http://0.0.0.0:8000/parse"):
    
    data = {"locale":"it_IT",
        "text":text}
    
    resp = None
    
    datetimes = list()
    
    try:
    
        response = requests.post(url, data=data)
        
        try:
        
            if response.status_code == 200:

                for dt in response.json():
                    
                    if dt['dim'] == "time":

                        dtime = dt['value']['value']
                        dtime = dateparser.parse(dtime)

                        datetimes.append(dtime)

            resp = dict()
            
            if len(datetimes) > 1:
                resp['datetime'] = list([min(datetimes), max(datetimes)])
            else:
                resp['datetime'] = list(datetimes)
            
        except:
            pass
        
    except:
        pass
    
    
    return resp

In [80]:
def add_slots(sentence: str, model: sklearn_crfsuite.estimator.CRF) -> dict:

    """
    Integrazione dictionary - parte 2 di 2
    """

    sentence_dict = {}

    sentence = rimuovi_punteggiatura(sentence)

    slots_dict = slots_extractor(sentence, model)

    sentence_dict["sentence"] = sentence
    sentence_dict["slots"] = slots_dict

    sentence_dict = replace_slot_values(sentence_dict)

    #     date_time = extract_datetime(sentence)

    #     if date_time is not None and len(date_time) > 0:
    #         sentence_dict['datetimes'] = date_time['datetime']
    #     else:
    #         sentence_dict['datetimes'] = False

    return sentence_dict

In [81]:
add_slots('CI SONO FATTORIE DIDATTICHE, AD AFRAGOLA CHE SONO APERTE DOMANI ALLE 8 DI SERA ?', crf)

{'sentence': 'CI SONO FATTORIE DIDATTICHE AD AFRAGOLA CHE SONO APERTE DOMANI ALLE 8 DI SERA ',
 'slots': defaultdict(list, {'PROVINCIA': 'SERA'}),
 'replaced_sentence': 'CI SONO FATTORIE DIDATTICHE AD AFRAGOLA CHE SONO APERTE DOMANI ALLE 8 DI PROVINCIA '}

In [82]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer()

In [83]:
cv.fit(df.sentences)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [84]:
print(f"Il vocabolario contiene {len(cv.vocabulary_)} parole")

cv.vocabulary_

Il vocabolario contiene 17 parole


{'quante': 14,
 'fattorie': 8,
 'didattiche': 7,
 'ci': 3,
 'sono': 16,
 'in': 9,
 'provincia': 13,
 'di': 6,
 'con': 5,
 'categoria': 1,
 'nella': 10,
 'che': 2,
 'producono': 11,
 'produzione': 12,
 'ad': 0,
 'sede': 15,
 'coltivazione': 4}

In [85]:
fake_list = list()

for i in range(30):
    fake_list.append(" ".join(random.choices(list(cv.vocabulary_.keys()), k=10)))

In [86]:
fake_list[:10]

['fattorie produzione in di nella sono fattorie nella produzione didattiche',
 'producono sede produzione ci fattorie sono sede con quante provincia',
 'sede quante sede di sede fattorie sono didattiche produzione di',
 'in sede didattiche provincia producono ad produzione quante di coltivazione',
 'di didattiche categoria provincia producono di coltivazione produzione ad di',
 'ci quante in produzione fattorie coltivazione fattorie quante ad sono',
 'nella in nella sono provincia sede sede che di produzione',
 'nella provincia didattiche fattorie didattiche ci con ad in produzione',
 'categoria coltivazione fattorie producono nella categoria didattiche in coltivazione categoria',
 'sono in producono fattorie provincia provincia ad categoria producono che']

In [87]:
df_fake = pd.DataFrame({"sentences": fake_list, "intents": "fake"})
df_fake.head()

,sentences,intents
0,fattorie produzione in di nella sono fattorie ...,fake
1,producono sede produzione ci fattorie sono sed...,fake
2,sede quante sede di sede fattorie sono didatti...,fake
3,in sede didattiche provincia producono ad prod...,fake
4,di didattiche categoria provincia producono di...,fake


In [88]:
df = pd.concat([df, df_fake], axis=0, ignore_index=True)
df.tail()

,sentences,intents
1025,didattiche quante ci ci ci sede ad ci in colti...,fake
1026,fattorie didattiche che produzione produzione ...,fake
1027,nella categoria didattiche coltivazione con pr...,fake
1028,quante ad didattiche sede con produzione di di...,fake
1029,producono nella sede ad con sono in ci con sono,fake


In [89]:
import sklearn

from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

In [90]:
df.intents.unique()

array(['user_AskProvWithParams', 'user_AskProv', 'user_AskCity', 'fake'],
      dtype=object)

In [91]:
le.fit(df.intents.unique())

for n, cls in enumerate(le.classes_):
    print(f"{cls} --> {n}")

fake --> 0
user_AskCity --> 1
user_AskProv --> 2
user_AskProvWithParams --> 3


In [92]:
labels_categories = le.transform(df.intents.values)

In [93]:
le.inverse_transform([1])[0]

'user_AskCity'

In [95]:
print(df.sentences[0])

np.max(cv.transform(df.sentences[0].split()).toarray(), axis=0)

QUANTE FATTORIE DIDATTICHE CI SONO IN PROVINCIA DI PROVINCIA CON CATEGORIA 


array([0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1])

In [96]:
from sklearn.linear_model import SGDClassifier

classifier = SGDClassifier(fit_intercept=False, loss="log", random_state=200)

In [97]:
classifier.fit(X=cv.transform(df["sentences"].values).toarray(), y=labels_categories)

SGDClassifier(alpha=0.0001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=False,
              l1_ratio=0.15, learning_rate='optimal', loss='log', max_iter=1000,
              n_iter_no_change=5, n_jobs=None, penalty='l2', power_t=0.5,
              random_state=200, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)

In [98]:
df["intents"].values[0]

'user_AskProvWithParams'

In [99]:
df["sentences"].values[0]

'QUANTE FATTORIE DIDATTICHE CI SONO IN PROVINCIA DI PROVINCIA CON CATEGORIA '

In [100]:
y_pred = classifier.predict(
    [np.max(cv.transform(df["sentences"].values[0].split()).toarray(), axis=0)]
)

In [101]:
le.inverse_transform(y_pred)[0]

'user_AskProvWithParams'

In [102]:
def get_intents_and_slots(
    sentence: str,
    model: sklearn_crfsuite.estimator.CRF,
    cv: sklearn.feature_extraction.text.CountVectorizer,
    le: sklearn.preprocessing._label.LabelEncoder,
    threasold=0.25,
) -> dict:

    """
    Estrae gli intents e gli slots dalla frase
    """

    sentence_dict = add_slots(sentence, crf)

    sentence = sentence_dict["replaced_sentence"]

    arr = cv.transform(add_slots(sentence, crf)["replaced_sentence"].split()).toarray()

    arr = np.max(arr, axis=0)

    probs = model.predict_proba([arr])[0]

    df = pd.DataFrame({"classes": le.classes_, "probs": probs})
    df.sort_values("probs", ascending=False, inplace=True)
    classes_with_prob = df[df["probs"] > threasold].to_dict("records")
    sentence_dict["intents"] = classes_with_prob

    max_intent = le.classes_[np.argmax(probs)]
    sentence_dict["max_intent"] = max_intent

    return sentence_dict

In [105]:
nuova_frase = "QUANTE FATTORIE DIDATTICHE CI SONO IN PROVINCIA DI CASERTA CON ALLEVAMENTO"

data = get_intents_and_slots(nuova_frase, classifier, cv, le)

In [106]:
data

{'sentence': 'QUANTE FATTORIE DIDATTICHE CI SONO IN PROVINCIA DI CASERTA CON ALLEVAMENTO',
 'slots': defaultdict(list,
             {'CATEGORIA': 'ALLEVAMENTO', 'PROVINCIA': 'CASERTA'}),
 'replaced_sentence': 'QUANTE FATTORIE DIDATTICHE CI SONO IN PROVINCIA DI PROVINCIA CON CATEGORIA',
 'intents': [{'classes': 'user_AskProvWithParams',
   'probs': 0.9995642844054845}],
 'max_intent': 'user_AskProvWithParams'}

In [107]:
dialogs = {v[0]: v[1] for v in list(dialogs.values())}

pprint.pprint(dialogs)

{'user_AskCity': 'bot_ReplyCity ',
 'user_AskProv': 'bot_ReplyProv',
 'user_AskProvWithParams': 'bot_ReplyProvWithParams'}


In [108]:
bot_sentences

defaultdict(list,
            {'bot_ReplyCity ': ['Nella città di [SEDE] ci sono %d fattorie didattiche'],
             'bot_ReplyProv': ['Nella provincia di [PROVINCIA] ci sono %d fattorie didattiche'],
             'bot_ReplyProvWithParams': ['Nella provincia di [PROVINCIA] ci sono %d fattorie didattiche con le caratteristiche richieste']})

In [109]:
"""
Estrazione intent
"""

if data["intents"][0]["probs"] > 0.75:

    intent = data["max_intent"]

else:

    intent = "fake"

intent

'user_AskProvWithParams'

In [110]:
"""
Estrazione risposta
"""

try:

    reply = dialogs[intent]

except:

    reply = "fake"

reply

'bot_ReplyProvWithParams'

Nella provincia di [PROVINCIA] ci sono %d fattorie didattiche con le caratteristiche richieste 

CATEGORIA == 'ALLEVAMENTO' and PROVINCIA == 'CASERTA'


### Il file "db_esempio" è stato ottenuto rielaborando un dataset scaricato sul portale degli OpenData disponibile al seguente [https://dati.regione.campania.it/catalogo/resources/Fattorie-didattiche.csv](https://dati.regione.campania.it/catalogo/resources/Fattorie-didattiche.csv)

In [120]:
df_query = pd.read_csv("db_esempio.csv")
df_query.head()

,NOME,SEDE,PROVINCIA,CATEGORIA
0,Azienda Sperimentale Regionale Improsta,EBOLI,SALERNO,ALLEVAMENTO
1,"Museo della Civiltà Contadina ""Michele Russo""",SOMMA VESUVIANA,NAPOLI,ALLEVAMENTO
2,Masseria Panico di Eredi Beneduce Ettore,SOMMA VESUVIANA,NAPOLI,ALLEVAMENTO
3,Cooperativa Agrituristica La Ginestra,VICO EQUENSE,NAPOLI,ALLEVAMENTO
4,Agriturismo Costiera Amalfitana,TRAMONTI,SALERNO,ALLEVAMENTO


In [123]:
"""
Estrazione frase con parametri
"""

if reply != "fake":

    reply_str = random.choice(bot_sentences[reply])

reply_str

print(reply_str, "\n")

"""
Estrazione dei dati per la sostituzione dei parametri
"""

query_list = list()

for k, v in data["slots"].items():
    query_list.append(f"{k} == '{v}'")

    k = list(data["slots"].keys())
    k = f"[{k}]"

    v = list(data["slots"].values())

    reply_str = re.sub(f"[{k}+]", v, reply_str)

query = " and ".join(query_list)

print(query)

Nella provincia di [PROVINCIA] ci sono %d fattorie didattiche con le caratteristiche richieste 



<ipython-input-123-374570fe609a>:27: FutureWarning: Possible nested set at position 1
  reply_str = re.sub(f"[{k}+]", v, reply_str)


TypeError: unhashable type: 'list'

In [121]:
query

"CATEGORIA == 'ALLEVAMENTO' and PROVINCIA == 'CASERTA'"

In [122]:
n = df_query.query(query)["NOME"].drop_duplicates().count()

print(reply_str % (n))

Nella provincia di [PROVINALLEVAMENTO ci sono 23 fattorie didattiche con le caratteristiche richieste


In [124]:
def bot_reply(
    sentence: str,
    model: sklearn_crfsuite.estimator.CRF,
    cv: sklearn.feature_extraction.text.CountVectorizer,
    le: sklearn.preprocessing._label.LabelEncoder,
    dialogs: dict,
    bot_sentences: dict,
    db: str = "db_esempio.csv",
    threasold=0.25,
) -> dict:

    """
    Genera la risposta
    """

    sentence = sentence.upper()

    data = get_intents_and_slots(sentence, model, cv, le, threasold)

    #     intent = data['max_intent']

    pprint.pprint(data)

    print("---------------------------------------------")

    if (data["intents"][0]["probs"] > 0.75) and data["intents"][0]["classes"] != "fake":

        intent = data["max_intent"]

        reply = dialogs[intent]

        reply_str = random.choice(bot_sentences[reply])

        query_list = list()

        for k, v in data["slots"].items():

            query_list.append(f"{k} == '{v}'")

            reply_str = re.sub(fr"\[{k}\]", v, reply_str)

        query = " and ".join(query_list)

        df_query = pd.read_csv(db)

        n = df_query.query(query)["NOME"].drop_duplicates().count()

        return reply_str % (n)

    else:

        intent = "Non ho capito, riformula meglio la tua domanda"

        return intent

In [126]:
nuova_frase = "Quante fattorie didattiche ci sono a Aversa"  # con allevamento ?"

bot_reply(nuova_frase, classifier, cv, le, dialogs, bot_sentences)

{'intents': [{'classes': 'user_AskCity', 'probs': 0.9983462899813043}],
 'max_intent': 'user_AskCity',
 'replaced_sentence': 'QUANTE FATTORIE DIDATTICHE CI SONO A SEDE',
 'sentence': 'QUANTE FATTORIE DIDATTICHE CI SONO A AVERSA',
 'slots': defaultdict(<class 'list'>, {'SEDE': 'AVERSA'})}
---------------------------------------------


'Nella città di AVERSA ci sono 0 fattorie didattiche'